In [1]:
!pip install qutip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 33.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
import builtins
from typing import List

# Set Up + Useful Functions

In [118]:
zero_ket = basis(2,0)
zero_rho = zero_ket * zero_ket.dag()
one_rho = basis(2,1) * basis(2,1).dag()
I = lambda: qeye(2)


def initial_rho(num_qubits):
  return tensor([zero_rho for i in range (num_qubits)])


def bell_state(state):
  """
    returns requested bell state
  """
  if state == 'phi+':
      phi_plus_ket = (tensor(basis(2, 0), basis(2, 0)) + tensor(basis(2, 1), basis(2, 1))).unit()
      return phi_plus_ket * phi_plus_ket.dag()

  elif state == 'phi-':
      phi_minus_ket = (tensor(basis(2, 0), basis(2, 0)) - tensor(basis(2, 1), basis(2, 1))).unit()
      return phi_minus_ket * phi_minus_ket.dag()

  elif state == 'psi+':
      psi_plus_ket = (tensor(basis(2, 0), basis(2, 1)) + tensor(basis(2, 1), basis(2, 0))).unit()
      return psi_plus_ket * psi_plus_ket.dag()

  elif state == 'psi-':
      psi_minus_ket = (tensor(basis(2, 0), basis(2, 1)) - tensor(basis(2, 1), basis(2, 0))).unit()
      return psi_minus_ket * psi_minus_ket.dag()

  else:
      raise ValueError("Invalid Bell state label")


def pad_op_to_left(op, num_identities):
  if num_identities > 0:
    left_op = I()
    for i in range(num_identities - 1):
      left_op = tensor(left_op, I())
    return tensor(left_op, op)
  else:
    return op


def pad_op_to_right(op, num_identities):
  if num_identities > 0:
    right_op = I()
    for i in range(num_identities - 1):
      right_op = tensor(right_op, I())
    return tensor(op, right_op)
  else:
    return op


def state_inserter(rho_initial, target_qubits, state):
  """
    Takes a density matrix of a network, inserts the state 'state' into the slots of the target_qubits.
    Assumes that target qubits are beside eachother and that no entanglement exists between
    qubits at either side of the target_qubits.

    Returns full density matrix of updated network
  """
  if isinstance(target_qubits, int):
    num_qubits_left = target_qubits
    num_qubits_right = num_qubits - target_qubits - 1

    qubits_left_list = [i for i in range(num_qubits_left)]
    qubits_right_list = [i + num_qubits_left + 1 for i in range(num_qubits_right)]

  elif isinstance(target_qubits, builtins.list):
    num_qubits_left = target_qubits[0]
    num_qubits_right = num_qubits - target_qubits[-1] - 1

    qubits_left_list = [i for i in range(num_qubits_left)]
    qubits_right_list = [i + num_qubits_left + len(target_qubits) for i in range(num_qubits_right)]

  else:
    raise TypeError('Invalid data type for target_qubit, must be int or list')

  rho = state
  if len(qubits_left_list) != 0:
    rho_qubits_left = rho_initial.ptrace(qubits_left_list)
    rho = tensor(rho_qubits_left, rho)

  if len(qubits_right_list) != 0:
    rho_qubits_right = rho_initial.ptrace(qubits_right_list)
    rho = tensor(rho, rho_qubits_right)

  return rho


def time_for_link(eta_eff, T_p):
  """
  Gets the time taken to generate entanglement between 2 adjacent nodes
  Does this by generating random samples from a geometric distribution with
  success probability eta_eff and then multiplying by time taken for one trial.

  Parameters:
  eta_eff (float): Success probability for the geometric distribution.
  num_samples (int): Number of random samples to generate.

  Returns:
  np.ndarray: Array of random samples from the geometric distribution.
  """
  no_of_trials = np.random.geometric(eta_eff)
  time_per_trial = T_p + 2 * d / c
  time_per_link = no_of_trials * time_per_trial
  return time_per_link


def dephasing_channel(rho, t, left_padding, right_padding):
  dp_prob = (1 - np.exp(-t / T_dp)) / 2
  dp_op = np.sqrt(1 - dp_prob) * qeye(2) + np.sqrt(dp_prob) * sigmaz()
  dp_op = pad_op_to_left(dp_op, left_padding)
  dp_op = pad_op_to_right(dp_op, right_padding)
  rho_t = dp_op * rho * dp_op.dag()
  return rho_t

def dark_counts_entang_gen(rho_initial, q1, q2, n, P_link):
  """
  q2 experiences the dark counts. q1 is included in the partial trace as
  entanglement is broken otherwise
  n = total probability that a pair is established = P_link * n_ch(d)
  NOTE: This only does one target_qubit
  """
  # channel efficiency
  n_ch = lambda L: np.exp(-L/L_att)

  # total probability that a pair is established
  n = P_link * n_ch(d)

  # the chance for a detector to click (including dark counts)
  n_eff = 1 - (1 - n) * (1 - p_d) ** 2

  # given a click occurs, the probability it is from a real event
  alpha = lambda n: (n * (1 - p_d)) / n_eff

  dark_count_rho = rho_initial.ptrace([q1, q2]) * alpha(n) + (1 - alpha(n)) / 2 * tensor(rho_initial.ptrace(q1) , I())
  rho = state_inserter(rho_initial = rho_initial,
                      target_qubits = [q1,q2], state = dark_count_rho)
  return rho


def initial_F_rho(rho_initial, target_qubits: List[int], F_initial: float):
  rho_bell = F_initial * bell_state('phi+') + ((1 - F_initial) / 3) * (bell_state('phi-') +
                                                                    bell_state('psi+') +
                                                                    bell_state('psi-'))
  rho = state_inserter(rho_initial=rho_initial, target_qubits=target_qubits, state=rho_bell)
  return rho

In [119]:
def entanglement_generation(rho_initial, target_qubits, F_initial, n, P_link, T_p):
  """
    This function generates entanglement across 2 neighbouring repeater stations
    q1 and q2. It does this by first creating a phi+ bell state density matrix
    across 2 qubits (with a fidelity F_initial). This is inserted into the initial
    state rho_initial by taking the partial trace of rho_initial to the left of the
    pair and to the right of the pair and then tensoring these density matrices together.

    (This function assumes that no entangled pair exists with one qubit on each
    side of the 2 selected qubits which I don't believe happens in any networking
    protocol)
  """
  # SET UP
  # getting the number of qubits either side of the pair and making a list of them
  q1, q2 = target_qubits[0], target_qubits[1]
  num_qubits_left = q1
  num_qubits_right = (num_qubits - 1) - q2
  qubits_left_list = [i for i in range(num_qubits_left)]
  qubits_right_list = [i + num_qubits_left+2 for i in range(num_qubits_right)]


  # GENERATING PHI+ BELL PAIR WITH FIDELITY F_INITIAL
  rho = initial_F_rho(rho_initial=rho_initial, target_qubits=[q1, q2], F_initial=F_initial)

  # ADDING MEMORY NOISE
  # dephasing channel for q1 (stays in memory for twice as long)
  rho = dephasing_channel(rho=rho, t = 2 * d/c, left_padding = num_qubits_left, right_padding = num_qubits_right + 1)

  # dephasing channel for q2
  rho = dephasing_channel(rho=rho, t = d/c, left_padding = num_qubits_left + 1, right_padding = num_qubits_right)

  # ADDING DARK COUNTS NOISE
  rho = dark_counts_entang_gen(rho_initial=rho, q1=q1, q2=q2, n=n, P_link=P_link)

  return rho

In [120]:
# DEFINING GLOBAL PARAMETERS
# Geometric Set Up
num_qubits = 4
total_L = 100
d = total_L/(num_qubits - 1)

# Optical Fiber Properties
L_att = 22e3       # attenuation length of optical fiber
c = 2e8            # speed of light in fiber optic

# Global Error Parameters
T_dp = 100         # dephasing time constant (memory error)
p_d = 0            # probability a dark count will occur in a detection window


# DEFINING ENTANGLEMENT GENERATION PARAMETERS
entang_gen_args = {
    # initial fidelity of the bell state generated
    'n' : 1,             # probability a link is established between 2 neighbouring repeater stations
    'P_link' : 1,        # probability a link is established without considering distance based losses
    'T_p' : 0            # time to prepare an entangled pair
}

In [121]:
# INITIALISING SYSTEM + RUNNNING ENTANGLEMENT GENERATION EVENT
rho = initial_rho(num_qubits)
rho = entanglement_generation(rho_initial=rho, target_qubits=[0,1], F_initial=1, **entang_gen_args)

for i in range(num_qubits):
  print(f'Qubit{i}\n', rho.ptrace(i).full(), '\n')
np.round(rho.ptrace([0,1]).full(), 3)

Qubit0
 [[0.5000697 +0.j 0.        +0.j]
 [0.        +0.j 0.49993031+0.j]] 

Qubit1
 [[0.5000697 +0.j 0.        +0.j]
 [0.        +0.j 0.49993031+0.j]] 

Qubit2
 [[1.00000001+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j]] 

Qubit3
 [[1.00000001+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j]] 



array([[0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
       [0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j]])

# Starting Entanglement Generation

In [122]:

def entanglement_swapping(rho_initial, q1q2, q3q4, lambda_BSM):
  """
  rho_initial = denisty matrix
  target_qubits = list of length 4
  """
  q0, q1 = q1q2[0], q1q2[1]
  q2, q3 = q3q4[0], q3q4[1]
  num_qubits_left = q1
  num_qubits_right = (num_qubits - 1) - q3
  qubits_left_list = [i for i in range(num_qubits_left)]
  qubits_right_list = [i + num_qubits_left+2 for i in range(num_qubits_right)]

  bell_measure_op = tensor(I(), bell_state('phi+'), I())

  # getting the entangled state of qubits 0 and 3
  rho_q1q2 = rho_initial.ptrace(q1q2)
  rho_q3q4 = rho_initial.ptrace(q3q4)
  rho_initial_target = tensor(rho_q1q2, rho_q3q4)

  state_03 = (bell_measure_op * rho_initial_target * bell_measure_op.dag()).ptrace([0,3]).unit()
  state_03 = lambda_BSM * state_03 + (1 - lambda_BSM) / 4 * tensor(I(), I())


  # tensoring with two 0 state qubits (their state doesnt matter bc we wont be using again, im saying 0 to keep it simple)
  state_0312 = tensor(state_03, zero_rho, zero_rho)

  perm_order = [0, 2, 3, 1]
  state = state_0312.permute(perm_order)
  rho = state_inserter(rho_initial=rho_initial, target_qubits=target_qubits, state=state)
  return rho

For fidelities

In [125]:
num_qubits = 4
total_L = 100
d = total_L/(num_qubits - 1)

rho = initial_rho(num_qubits)
rho = entanglement_generation(rho_initial=rho, target_qubits=[0,1], F_initial = 0.9, **entang_gen_args)
rho = entanglement_generation(rho_initial = rho, target_qubits=[2,3], F_initial = 0.75, **entang_gen_args)
rho = entanglement_swapping(rho_initial = rho, q1q2 = [0,1], q3q4=[2,3], lambda_BSM= 1)

#for i in range(num_qubits):
  #print(f'Qubit{i}\n', np.round(rho.ptrace(i).full(),3), '\n')

state_03 = rho.ptrace([0,3])
#print('Qubits 0 and 3 \n',np.round(state_03.full(), 3))

phi_plus_ket = (tensor(basis(2, 0), basis(2, 0)) + tensor(basis(2, 1), basis(2, 1))).unit()
print('\nFidelity:', np.round(phi_plus_ket.dag() * state_03 * phi_plus_ket, 3))

NameError: name 'target_qubits' is not defined

In [114]:
initial_ket = tensor(basis(2,0), basis(2,0),basis(2,0),basis(2,1),basis(2,1),basis(2,1))
initial_rho = initial_ket * initial_ket.dag()
reduced = initial_rho.ptrace([0,4,1])
for i in range(3):
  print(reduced.ptrace(i).full(),'\n')

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

[[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]] 

